In [ ]:
from __future__ import print_function
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
class MLP(nn.Module):
    def __init__(self, node_list):
        '''
        params:
        node_list (int list): n elements where the first element is the input layers' node number,
                                the middle n-1 elements are hidden layers' node numbers,
                                and the last element is the output layers' node number
        '''
        super(MLP, self).__init__()
        self.classifier = self._make_layers(node_list)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.classifier(x)
        return out

    def _make_layers(self, node_list):
        layers = []
        in_dim = node_list[0]
        for idx in range(1, len(node_list) - 1):
            num = node_list[idx]
            layers += [AdvancedDropout(in_dim),
                        nn.Linear(in_dim, num),
                        nn.ReLU(inplace=True)]
            in_dim = num
        layers += [AdvancedDropout(in_dim),
                    nn.Linear(in_dim, node_list[-1])]
        return nn.Sequential(*layers)

In [ ]:
import torch
import numpy as np
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter
import torch.nn.functional as F

class AdvancedDropout(Module):

    def __init__(self, num, init_mu=0, init_sigma=1.2, reduction=16 ): #Change this expirement 1#
        '''
        params:
        num (int): node number
        init_mu (float): intial mu
        init_sigma (float): initial sigma
        reduction (int, power of two): reduction of dimention of hidden states h
        '''
        super(AdvancedDropout, self).__init__()
        if init_sigma <= 0:
            raise ValueError("Sigma has to be larger than 0, but got init_sigma=" + str(init_sigma))
        self.init_mu = init_mu
        self.init_sigma = init_sigma

        self.weight_h = Parameter(torch.rand([num // reduction, num]).mul(0.01))
        self.bias_h = Parameter(torch.rand([1]).mul(0.01))

        self.weight_mu = Parameter(torch.rand([1, num // reduction]).mul(0.01))
        self.bias_mu = Parameter(torch.Tensor([self.init_mu]))
        self.weight_sigma = Parameter(torch.rand([1, num // reduction]).mul(0.01))
        self.bias_sigma = Parameter(torch.Tensor([self.init_sigma]))

    def forward(self, input):
        if self.training:
            c, n = input.size()
            # parameterized prior
            h = F.linear(input, self.weight_h, self.bias_h)
            mu = F.linear(h, self.weight_mu, self.bias_mu).mean()
            sigma = F.softplus(F.linear(h, self.weight_sigma, self.bias_sigma)).mean()
            # mask
            epsilon = mu + sigma * torch.randn([c, n]).cuda()
            mask = torch.sigmoid(epsilon)

            out = input.mul(mask).div(torch.sigmoid(mu.data / torch.sqrt(1. + 3.14 / 8. * sigma.data ** 2.))) #added alpha and change function in expirement 3
        else:
            out = input

        return out

In [ ]:
def toggle_dropout(net, epoch, interval=20): #Control the intervals for expiremnet 2
    is_dropout_active = (epoch // interval) % 2 == 0
    for module in net.classifier.classifier:
        if isinstance(module, AdvancedDropout):
            module.training = is_dropout_active
            print('Dropout: ' + str(is_dropout_active))

In [ ]:
def main():
 #os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # use GPU 0
    nb_epoch = 300  # number of epochs
    lr = 0.1  # initial learning rate
    dropout_epochs = int(nb_epoch * 0.2)  # first 20% of epochs
    train_accuracies = []
    train_losses = []
    test_accuracies = []
    test_losses = []



    print("OPENING " + 'results_train.csv')
    results_train_file = open('results_train.csv', 'a')
    results_train_file.write('epoch,train_acc,train_loss\n')
    results_train_file.flush()

    print("OPENING " + 'results_test.csv')
    results_test_file = open('results_test.csv', 'a')
    results_test_file.write('epoch,test_acc,test_loss\n')
    results_test_file.flush()

    # Data
    print('==> Preparing data..')
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root='/data/xiejiyang/data',  # your data path
        train=True,
        download=True,
        transform=transform_train
    )
    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=256,
        shuffle=True,
        num_workers=4
    )

    testset = torchvision.datasets.CIFAR10(
        root='/data/xiejiyang/data',  # your data path
        train=False,
        download=True,
        transform=transform_test
    )
    testloader = torch.utils.data.DataLoader(
        testset,
        batch_size=256,
        shuffle=False,
        num_workers=4
    )

    # Model
    print('==> Building model..')
    net = models.vgg16_bn(pretrained=False)
    net = nn.Sequential(*list(net.children())[:-2])  # get vgg16 backbone (only all the conv layers)

    class model_vgg(nn.Module):
        def __init__(self, model, node_list):
            '''
            params:
            model (nn.Sequential): backbone
            node_list (int list): n elements where the first element is the input layers' node number,
                                    the middle n-1 elements are hidden layers' node numbers,
                                    and the last element is the output layers' node number
            '''
            super(model_vgg, self).__init__()
            self.features = model
            self.classifier = MLP(node_list)  # fc layers

        def forward(self, x):
            x = self.features(x)
            x = self.classifier(x)
            return x

    net = model_vgg(net, [512*1*1, 512, 10])  # construct the model

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = net.to(device)
    net.features.to(device)
    net.classifier.to(device)

    criterion = nn.CrossEntropyLoss()

    # set optimizer
    dp_params = []
    res_params = []
    for m in net.classifier.classifier:
        if isinstance(m, AdvancedDropout):
            dp_params.append(m.weight_h)
            dp_params.append(m.bias_h)
            dp_params.append(m.weight_mu)
            dp_params.append(m.bias_mu)
            dp_params.append(m.weight_sigma)
            dp_params.append(m.bias_sigma)
        elif isinstance(m, nn.Linear):
            res_params.append(m.weight)
            if hasattr(m, "bias"):
                res_params.append(m.bias)

    optimizer = optim.SGD([
        {'params': net.features.parameters(), 'lr': lr},
        {'params': res_params, 'lr': lr},
        {'params': dp_params, 'lr': 1e-4}
    ], momentum=0.9, weight_decay=5e-4)

    def train(epoch):
        # Toggle dropout layers based on the current epoch
        toggle_dropout(net, epoch)

        net.train()
        train_loss = 0
        correct = 0
        total = 0
        idx = 0

        for batch_idx, (inputs, targets) in enumerate(trainloader):
            idx = batch_idx
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.detach().cpu()
            _, predicted = torch.max(outputs, -1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).sum().item()

        train_acc = 100. * correct / total
        train_loss = train_loss / (idx + 1)
        print('Iteration %d, train_acc = %.4f, train_loss = %.4f' % (epoch, train_acc, train_loss))
        results_train_file.write('%d,%.4f,%.4f\n' % (epoch, train_acc, train_loss))
        results_train_file.flush()

        train_accuracies.append(train_acc)
        train_losses.append(train_loss)

        return train_acc, train_loss


    def test(epoch):
        net.eval()
        test_loss = 0
        correct = 0
        total = 0
        idx = 0
        for batch_idx, (inputs, targets) in enumerate(testloader):
            idx = batch_idx
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)

            loss = criterion(outputs, targets)

            test_loss += loss.detach().cpu()
            _, predicted = torch.max(outputs, -1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).sum().item()

        test_acc = 100. * correct / total
        test_loss = test_loss / (idx + 1)
        print('Iteration %d, test_acc = %.4f, test_loss = %.4f' % (epoch, test_acc, test_loss))
        results_test_file.write('%d,%.4f,%.4f\n' % (epoch, test_acc, test_loss))
        results_test_file.flush()

        test_accuracies.append(test_acc)
        test_losses.append(test_loss)

        return test_acc, test_loss

    for epoch in tqdm(range(0, nb_epoch)):
      print('\nEpoch: %d' % epoch)
      if epoch in [75, 110]:  # learning rate drop
        for param_group in optimizer.param_groups:
            param_group['lr'] /= 10
      train_acc, train_loss = train(epoch)
      test_acc, test_loss = test(epoch)
      torch.save(net.state_dict(), 'checkpoint.pth')

    # Plotting results
    plt.figure(figsize=(12, 5))

    # Plot training and test accuracy
    plt.subplot(1, 2, 1)
    plt.plot(range(nb_epoch), train_accuracies, label='Train Accuracy')
    plt.plot(range(nb_epoch), test_accuracies, label='Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Test Accuracy')

    # Plot training and test loss
    plt.subplot(1, 2, 2)
    plt.plot(range(nb_epoch), train_losses, label='Train Loss')
    plt.plot(range(nb_epoch), test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Test Loss')

    plt.tight_layout()
    plt.show()

In [ ]:
main()

OPENING results_train.csv
OPENING results_test.csv
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  0%|          | 0/300 [00:00<?, ?it/s]


Epoch: 0
Dropout: True
Dropout: True
Iteration 0, train_acc = 35.4520, train_loss = 1.7039


  0%|          | 1/300 [00:12<1:00:41, 12.18s/it]

Iteration 0, test_acc = 44.6300, test_loss = 1.4569

Epoch: 1
Dropout: True
Dropout: True
Iteration 1, train_acc = 55.4300, train_loss = 1.2637


...

Iteration 232, test_acc = 93.7600, test_loss = 0.3156

Epoch: 233
Dropout: False
Dropout: False
Iteration 233, train_acc = 99.9840, train_loss = 0.0012


 78%|███████▊  | 234/300 [47:54<13:28, 12.24s/it]

Iteration 233, test_acc = 93.7900, test_loss = 0.3153

Epoch: 234
Dropout: False
Dropout: False
Iteration 234, train_acc = 99.9900, train_loss = 0.0012


 78%|███████▊  | 235/300 [48:06<13:14, 12.22s/it]

Iteration 234, test_acc = 93.7500, test_loss = 0.3160

Epoch: 235
Dropout: False
Dropout: False
